In [543]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
from datetime import datetime

In [673]:
driver = webdriver.Edge()
driver.get('https://www.linkedin.com/login')

In [674]:
username = "shreyashthengne@gmail.com"
pw = "Qwerasdf@2003&"

In [675]:
driver.find_element(By.ID, "username").send_keys(username)
driver.find_element(By.ID, "password").send_keys(pw)
driver.find_element(By.CLASS_NAME, "login__form_action_container").click()

# Extracting user profile data

In [676]:
driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/div/div/div[1]/div[1]/a').click()

In [677]:
def nested_list_to_dict(l):
    d = {}
    for i in l:
        d[i[0]] = i[1]
    return d

Get Contact INFO

In [678]:
name_related_data = driver.find_element(By.CLASS_NAME, "YFSfxlnCoALrcxZcdsGecfbthcLuxrPpFA").text.split('\n')
name = name_related_data[0]
pronouns = name_related_data[1]

driver.find_element(By.ID, "top-card-text-details-contact-info").click()
obj_list = WebDriverWait(driver, 10).until(ec.presence_of_all_elements_located((By.CLASS_NAME, "pv-contact-info__contact-type")))
contact_info = []
for sel_obj in obj_list:
    contact_info.append(sel_obj.text.split("\n"))
try:
    contact_info.remove(['Birthday'])
except:
    pass

actions = ActionChains(driver)
actions.send_keys(Keys.ESCAPE).perform()
contact_info_dict = nested_list_to_dict(contact_info)
contact_info_dict['name'] = name
contact_info_dict['pronouns'] = pronouns
# contact_info_dict

Getting About Section Text

In [679]:
actions = ActionChains(driver)

In [680]:
about = driver.find_elements(By.CLASS_NAME, "UBNjRtiGpOBMCdHNiTtJimrddrsdVzA")[1].text

In [681]:
# connections = driver.find_element(By.ID, 'ember450').text
driver.find_elements(By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/ul')

[<selenium.webdriver.remote.webelement.WebElement (session="1edabd1e7f698002982f2ca12c6fa987", element="f.273404F8415425DC30F76A46288C7785.d.FB8F5D19512BD28D6E4CFBE872B62179.e.242")>]

In [682]:
driver.find_element(By.ID, 'global-nav-search').click()
actions.send_keys("amazon sde").send_keys(Keys.ENTER).perform()
WebDriverWait(driver, 10).until(ec.presence_of_all_elements_located((By.CLASS_NAME, "search-reusables__primary-filter")))[0].click()
actions.send_keys(Keys.ESCAPE).perform()

In [685]:
driver.find_element(By.CLASS_NAME, 'reusable-search__result-container').click()

In [696]:
name_related_data = driver.find_element(By.CLASS_NAME, "YFSfxlnCoALrcxZcdsGecfbthcLuxrPpFA").text.split('\n')
name = name_related_data[0]

driver.find_element(By.ID, "top-card-text-details-contact-info").click()
obj_list = WebDriverWait(driver, 10).until(ec.presence_of_all_elements_located((By.CLASS_NAME, "pv-contact-info__contact-type")))
contact_info = []
for sel_obj in obj_list:
    contact_info.append(sel_obj.text.split("\n"))
try:
    contact_info.remove(['Birthday'])
except:
    pass
    

actions = ActionChains(driver)
actions.send_keys(Keys.ESCAPE).perform()
contact_info_dict = nested_list_to_dict(contact_info)
contact_info_dict['name'] = name

In [697]:
contact_info_dict

{'abhishek’s Profile': 'linkedin.com/in/abhishek-paithane-a617a3119',
 'name': 'abhishek paithane'}

In [563]:
source_code = driver.page_source
soup = BeautifulSoup(source_code, 'html.parser')

In [564]:
connections = soup.find('ul', {'class', "AVBFDyqTfsuTgIZXdCfRksCUrxiXXUGBgrQ ARBouGUeqZjIUscRznwDQyGWDCzezcaOPhbnA"}).text.strip()
followers = soup.find('p', {'class', 'rIGQerBSNRmmmzDMQKpqMZqBKoJtRwJlYno pvs-header__optional-link text-body-small'}).text.strip().split('followers')[0][:-1]

about = soup.find('div', {'class', "display-flex ph5 pv3"}).text.strip()

In [619]:
cards = soup.find_all('section', {"class": "artdeco-card pv-profile-card break-words mt2"})

Experience

In [620]:
# try:
# except:
#     pass
for i in cards:
    if i.find('div', {'id':'experience'}) != None:
        all_exp = i
        break
all_exp = all_exp.find_all('li', {'class', 'artdeco-list__item MSekCOdsooFvletAzszqqmePbnkuYRoes YzGwfSNcLHPUoUUgVvMgQPXaOYiNMqbzVPKTGC'})

def extract_exp_data(sample):
    '''
    role, company, job_type, from, to, duration
    '''
    role = sample.find('div', {'class', 'display-flex flex-wrap align-items-center full-height'}).text.strip()
    role = role[:len(role)//2]

    company_and_job_type = sample.find('span', {'class', 't-14 t-normal'}).text.strip()
    company_and_job_type = company_and_job_type[:len(company_and_job_type)//2].split(" · ")
    company = company_and_job_type[0]
    # print(company_and_job_type)
    job_type = company_and_job_type[1]

    period, duration = sample.find('span', {'class', "pvs-entity__caption-wrapper"}).text.split(' · ')
    period = period.split(" - ")
    
    try:
        duration = int(duration.split(" ")[0]) * 12 + int(duration.split(" ")[2])
    except:
        duration = int(duration.split(" ")[0])

    try:
        period = {'from': datetime.strptime(period[0], "%b %Y"), 'to': datetime.strptime(period[1], "%b %Y")}
    except:
        period = {'from': datetime.strptime(period[0], "%b %Y"), 'to': datetime.strptime(datetime.now().strftime("%b %Y"), "%b %Y")}
    return {'role':role, 'company':company, 'job_type':job_type, 'from':period['from'], 'to':period['to'], 'duration':duration}

all_experiences = []
for i in  all_exp:
    all_experiences.append(extract_exp_data(i))

In [621]:
all_experiences

[{'role': 'SDE',
  'company': 'Amazon',
  'job_type': 'Full-time',
  'from': datetime.datetime(2022, 7, 1, 0, 0),
  'to': datetime.datetime(2024, 4, 1, 0, 0),
  'duration': 22},
 {'role': 'Software Development Engineer',
  'company': 'Swiggy',
  'job_type': 'Full-time',
  'from': datetime.datetime(2021, 12, 1, 0, 0),
  'to': datetime.datetime(2022, 6, 1, 0, 0),
  'duration': 7},
 {'role': 'Software Development Engineer',
  'company': 'AppDirect',
  'job_type': 'Full-time',
  'from': datetime.datetime(2021, 7, 1, 0, 0),
  'to': datetime.datetime(2021, 12, 1, 0, 0),
  'duration': 6},
 {'role': 'Tavisca, A cxLoyalty company',
  'company': 'Full-time',
  'job_type': '2 yrs 1 mo',
  'from': datetime.datetime(2020, 7, 1, 0, 0),
  'to': datetime.datetime(2021, 7, 1, 0, 0),
  'duration': 13}]

Education

In [617]:
# all_edu = soup.find_all('div', {'class', 'pvs-list__outer-container'})[7].find_all('li')
for i in soup.find_all('section', {"class": "artdeco-card pv-profile-card break-words mt2"}):
    if i.find('div', {'id':'education'}) != None:
        all_edu = i
        break


all_edu = all_edu.find_all('li', {'class', 'artdeco-list__item MSekCOdsooFvletAzszqqmePbnkuYRoes YzGwfSNcLHPUoUUgVvMgQPXaOYiNMqbzVPKTGC'})

def extract_education_data(sample):
    '''
    college, degree, period
    '''
    college = sample.find('div', {'class', 'display-flex flex-wrap align-items-center full-height'}).text.strip()
    college = college[:len(college)//2]
    degree = sample.find('span', {'class', 't-14 t-normal'}).text.strip()
    degree = degree[:len(degree)//2]
    period = sample.find('span', {'class', 'pvs-entity__caption-wrapper'}).text.strip().split(" - ")
    period = {'from': period[0], 'to': period[1]}
    return {'college':college, 'degree':degree, 'from':period['from'], 'to':period['to']}

all_education = []
for i in  all_edu:
    all_education.append(extract_education_data(i))
all_education

[{'college': "Pimpri Chinchwad Education Trust'S. Pimpri Chinchwad College Of Engineering",
  'degree': 'Bachelor of Engineering, Computer Software Engineering',
  'from': '2015',
  'to': '2019'},
 {'college': 'Maratha Highschool nasik',
  'degree': 'ssc, highschool studies',
  'from': '2008',
  'to': '2013'}]

Certifications

In [631]:

def extract_certifications(sample):
    '''
    name, org, issued, link
    '''
    sample = all_certs[0]
    name = sample.find('div', {'class', 'display-flex flex-wrap align-items-center full-height'}).text.strip()
    organization = sample.find('span', {'class', 't-14 t-normal'}).text
    organization = organization[:len(organization)//2]
    issued = datetime.strptime(sample.find('span', {'class', 'pvs-entity__caption-wrapper'}).text.split("Issued")[1].strip(), '%b %Y')
    link = sample.find('a', {'class', 'optional-action-target-wrapper artdeco-button artdeco-button--secondary artdeco-button--standard artdeco-button--2 artdeco-button--muted inline-flex justify-center align-self-flex-start button-placement-wrap'}).get('href')
    return {'cert_name':name, 'cert_org': organization, 'cert_issued':issued, 'cert_link':link}

try:
    driver.find_element(By.ID, 'navigation-index-see-all-licenses-and-certifications').click()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    all_certs = soup.find_all('li', {'class', 'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column'})

except:
    for i in cards:
        if i.find('div', {'id':'licenses_and_certifications'}) != None:
            all_certs = i
            break
    all_certs = all_certs.find_all('li', {'class', 'artdeco-list__item MSekCOdsooFvletAzszqqmePbnkuYRoes YzGwfSNcLHPUoUUgVvMgQPXaOYiNMqbzVPKTGC'})
    all_certs
all_certification = []
for i in  all_certs:
    all_certification.append(extract_certifications(i))

Skills

In [665]:
# def extrcat_skills(sample):
driver.get(driver.current_url+'details/skills/')
soup = BeautifulSoup(driver.page_source, 'html.parser')
all_skills = soup.find_all('li', {'class', 'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column'})
def extract_skills(sample):
    '''
    skill
    '''
    skill = sample.find('div', {'class':"display-flex flex-wrap align-items-center full-height"}).text.strip()
    skill = skill[:len(skill)//2]
    return skill

skills = []
for i in all_skills:
    skills.append(extract_skills(i))
skills = list(set(skills))
skills

['Software Design Patterns',
 'Object-Oriented Programming (OOP)',
 'React.js',
 'javascript',
 'HTML',
 'java',
 'c++ language programming',
 'Spring Webflux',
 'computer networks',
 'C++',
 'C',
 'Microservices',
 'Operating system',
 'Angular',
 'Low-Level Design',
 'Spring Boot',
 'Algorithms',
 'Java',
 'Spring Framework',
 'competitive programming',
 'design patterns',
 'High-Level Design',
 'Data Structures']